In [1]:
from dotenv import dotenv_values
from langchain.llms import OpenAI
config = dotenv_values(".env")
OPENAI_KEY = config["OPENAI_KEY"]

In [2]:
llm = OpenAI(openai_api_key=OPENAI_KEY)
print(llm.predict("what are some fun facts about mt everest?"))



1. Mount Everest stands at an incredible 8,848 meters (29,029 ft) tall, making it the highest mountain in the world.

2. Mount Everest is located in the Mahalangur Himal sub-range of the Himalayas in Nepal and Tibet.

3. The first successful summit of Everest was completed by Edmund Hillary and Tenzing Norgay in 1953.

4. The peak was named after Sir George Everest, a British surveyor-general of India.

5. Mount Everest has two main climbing routes, the southeast ridge from Nepal and the north ridge from Tibet.

6. The temperature at the summit of Mount Everest can drop to as low as -76°F (-60°C).

7. The lowest point on Mount Everest is the Western Cwm at 17,600 feet (5,364 meters).

8. The average altitude of Mount Everest is 26,000 feet (8,000 meters).

9. Mount Everest is home to some of the most extreme conditions on the planet, with high winds and freezing temperatures.

10. There are over 200 bodies on the mountain, most of which remain frozen in place.


## need document source(s) to proceed